# Load crypto market data

In [1]:
!git clone https://github.com/binance/binance-public-data.git

Cloning into 'binance-public-data'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 273 (delta 96), reused 49 (delta 49), pack-reused 150 (from 1)
Receiving objects: 100% (273/273), 6.50 MiB | 25.42 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [4]:
fromDate='2017-08-01'
toDate='2024-10-30'

In [5]:
!python3 binance-public-data/python/download-kline.py -t spot -s BTCUSDT -i 1h -startDate 2017-08-01 -endDate 2024-10-30 -skip-daily 1 -folder ~/Comp4651_Project/marketdata-1h/btc/

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

File Download: /home/sagemaker-user/Comp4651_Project/marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-08.zip
[##################################################]
File Download: /home/sagemaker-user/Comp4651_Project/marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-09.zip
[##################################################]
File Download: /home/sagemaker-user/Comp4651_Project/marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-10.zip
[##################################################]
File Download: /home/sagemaker-user/Comp4651_Project/marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-11.zip
[##################################################]
File Download: /home/sagemaker-user/Comp4651_Project/marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-0

Make sure the `marketdata` folder is in the same directory as this file for the code below to work

In [11]:
period = "1h"

In [12]:
import zipfile, os

def unzipFiles(sym,ticker):
    working_directory = './marketdata-{}/'.format(period)+sym+'/data/spot/monthly/klines/'+ticker+'/{}/'.format(period)+fromDate+'_'+toDate+'/'
    for file in os.listdir(working_directory):   # get the list of files
        print(file)
        if zipfile.is_zipfile(working_directory+file): # if it is a zipfile, extract it
            with zipfile.ZipFile(working_directory+file) as item: # treat the file as a zip
                print(item)
                item.extractall('marketdata-{}/'.format(period)+sym)

unzipFiles('btc','BTCUSDT')
unzipFiles('eth','ETHUSDT')

BTCUSDT-1h-2017-08.zip
<zipfile.ZipFile filename='./marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-08.zip' mode='r'>
BTCUSDT-1h-2017-09.zip
<zipfile.ZipFile filename='./marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-09.zip' mode='r'>
BTCUSDT-1h-2017-10.zip
<zipfile.ZipFile filename='./marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-10.zip' mode='r'>
BTCUSDT-1h-2017-11.zip
<zipfile.ZipFile filename='./marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-11.zip' mode='r'>
BTCUSDT-1h-2017-12.zip
<zipfile.ZipFile filename='./marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2017-12.zip' mode='r'>
BTCUSDT-1h-2018-01.zip
<zipfile.ZipFile filename='./marketdata-1h/btc/data/spot/monthly/klines/BTCUSDT/1h/2017-08-01_2024-10-30/BTCUSDT-1h-2018-01.zip' mode='r'>
BTCUSDT-1h-2018-02.zip
<zipf

In [1]:
S3_BUCKET=!(aws cloudformation list-exports --query "Exports[?Name=='public-blockchain-marketdata'].Value" --output text)
S3_BUCKET=S3_BUCKET[0]

In [2]:
!pip install awswrangler

In [3]:
folder = "marketdata-1h"

In [4]:
import zipfile, os
import boto3
import pandas as pd
import awswrangler as wr
from datetime import datetime

s3_path='s3://'+S3_BUCKET+"/marketdata/crypto"

def importFiles(sym,ticker):
    for file in os.listdir(folder+"/"+sym+"/"):   # get the list of files
        if file.endswith(".csv"): # check for ".zip" extension
            file_name = folder+"/"+sym+'/'+file # get full path of files
            #print(file_name)
            x=file_name.split('/')[-1:][0].split(".")[0]
            #print(x)
            
            hData=['open_time','open','high','low','close','vol','close_time','quote_asset_vol','number_of_trades','taker_base_asset_vol','taker_quote_asset_vol','ignore']
            df=pd.read_csv(file_name,header=0,names=hData,parse_dates=True)
            df['open_time'] = [datetime.fromtimestamp(x/1000) for x in df['open_time']]
            df['close_time'] = [datetime.fromtimestamp(x/1000) for x in df['close_time']]
            df['ticker']=ticker
            df.set_index(pd.DatetimeIndex(df['open_time']), inplace=True)
       
            s3_file=s3_path+'/'+x+'.snappy.parquet'
            print("upload to %s" % s3_file)
            r=wr.s3.to_parquet(
                    df=df,
                    path=s3_file,
                    dataset=False
            )

importFiles('btc','BTCUSDT')

upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2017-10.snappy.parquet


2024-11-27 14:05:10,564	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 411021312 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=0.95gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-11-27 14:05:11,759	INFO worker.py:1786 -- Started a local Ray instance.


upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2017-08.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2017-11.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2017-09.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2017-12.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2018-02.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2018-01.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2018-03.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto/BTCUSDT-1h-2018-04.snappy.parquet
upload to s3://crypto-marketdata-marketdatabucket-m13jkiz8o4p7/marketdata/crypto